# Coursera_Capstone
This project is part of IBM Data Science Professional Certification provided by Coursera

## 1. Introduction
This project is part of IBM Data Science Professional Certification provided by Coursera and consists in apply all the topics teached during the course to analyse and solve a real world study case.

The data was picked from http://data.dft.gov.uk/road-accidents-safety-data/DfTRoadSafety_Accidents_2019.zip and consist and contain detailed road safety data about the circumstances of personal injury road accidents in GB from 1979, the types of vehicles involved and the consequential casualties. The statistics relate only to personal injury accidents on public roads that are reported to the police, and subsequently recorded, using the STATS19 accident reporting form.

## 2. Methodology
This project applies the concept of CRISP-DM mmethodology  that is a process aimed at increasing the use of data mining over a wide variety of business applications and industries.

#### Why?
CRISP-DM is a highly flexible and cyclical model. Flexibility is required at each step along with communication to keep the project on track. It allows to  revisit and make adjusts in earlier stages if necessary. The key point of this process is that it’s cyclical.

CRISP-DM has 6 steps, namely:

<ul>  
 <li>
    <b>Business Understanding</b>
</li>  
 <li>
    <b>Data Understanding</b>
 </li>  
 <li>
     <b>Data Preparation</b>
 </li>  
 <li>
    <b>Modeling</b>
 </li>  
 <li>
    <b>Evaluation</b>
 </li> 
 <li>
    <b>Deployment</b>
 </li> 
</ul> 

### 2.1 Business Understanding

### 2.2. Data Understanding

### 2.3 Data Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
#from sklearn.metrics import jaccard_similarity_score
%matplotlib inline
import seaborn as sns

### Loading Data

In [2]:
#!wget -O data.csv "https://opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv"

In [3]:
df_raw = pd.read_csv('data.csv')
df_raw.columns.values

array(['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS',
       'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
       'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES',
       'SERIOUSINJURIES', 'FATALITIES', 'INCDATE', 'INCDTTM',
       'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKKEY', 'HITPARKEDCAR'], dtype=object)

<b>About data...</b><br>
1 - There is 221738 records in 40 columns and not all of columns have the same number of records.<br>
2 - The data contain categorical and continuos values distribuided in float64(5), int64(12), object(23).

<b> Balancing data </b> <br>
During Data Understanting some other projects were studied and was possible to understand wich kind of variable can be useful in this kind of analysis and that is good  before start balancing the data decide first  columns that contain information that apparently can be useful for the analysis goals.

Observing the data is notable tha there are columns that represent CODES that are irrelevant to our goals such: <code>OBJECTID</code>, <code>INCKEY</code>, <code>INTKEY</code>, <code>SDOT_COLCODE</code>, <code>SEGLANEKEY</code>, <code>INATTENTIONIND</code> and others keys <br>

In [4]:
df = df_raw[['X','Y','SEVERITYCODE','COLLISIONTYPE','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT',
            'VEHCOUNT','INJURIES','SERIOUSINJURIES','FATALITIES','WEATHER','ROADCOND','SPEEDING','JUNCTIONTYPE','UNDERINFL','LIGHTCOND']]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221738 entries, 0 to 221737
Data columns (total 17 columns):
X                  214260 non-null float64
Y                  214260 non-null float64
SEVERITYCODE       221737 non-null object
COLLISIONTYPE      195287 non-null object
PERSONCOUNT        221738 non-null int64
PEDCOUNT           221738 non-null int64
PEDCYLCOUNT        221738 non-null int64
VEHCOUNT           221738 non-null int64
INJURIES           221738 non-null int64
SERIOUSINJURIES    221738 non-null int64
FATALITIES         221738 non-null int64
WEATHER            195097 non-null object
ROADCOND           195178 non-null object
SPEEDING           9936 non-null object
JUNCTIONTYPE       209759 non-null object
UNDERINFL          195307 non-null object
LIGHTCOND          195008 non-null object
dtypes: float64(2), int64(7), object(8)
memory usage: 28.8+ MB


In [5]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df= missing_df[missing_df[0]>0]
missing_df

,index,0
0,X,7478
1,Y,7478
2,SEVERITYCODE,1
3,COLLISIONTYPE,26451
11,WEATHER,26641
12,ROADCOND,26560
13,SPEEDING,211802
14,JUNCTIONTYPE,11979
15,UNDERINFL,26431
16,LIGHTCOND,26730


#### Handling with missing values <br>

During data understanding was noticed that:<br>
1 - The values of column <code>SPEEDING</code> behave as boolean where value <code>Y</code> represents <code>TRUE</code> and  <code>(empty)</code> represents <code>FALSE</code>.<br>
2 - The values of <code>SEVERITYCODE</code>  can be <code>1,2,3,2b</code> and value <code>0</code> means unknown<br>
3 - All columns with values <code>Y</code> and <code>N</code> have boolean behave<br>
4 - Columns use values <code>Unknown</code> and <code>Other</code> also to express missing information.<br>

<br>
All this finding was applied before start droping data.

In [9]:
df["SEVERITYCODE"].replace('0', np.nan, inplace=True)
df["SPEEDING"].replace(np.nan,0 , inplace=True)
df.replace("Y", 1 , inplace=True)
df.replace("N", 0 , inplace=True)
df.replace("Unknown", np.nan , inplace=True)
df.replace("Other", np.nan , inplace=True)

In [7]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df= missing_df[missing_df[0]>0]
missing_df

,index,0
0,X,7478
1,Y,7478
2,SEVERITYCODE,21657
3,COLLISIONTYPE,51052
11,WEATHER,42632
12,ROADCOND,41835
14,JUNCTIONTYPE,12000
15,UNDERINFL,26431
16,LIGHTCOND,40507


In [11]:
print('Before', df.shape)
df.dropna(axis=0, inplace=True)
print('After', df.shape)

Before (148171, 17)
After (148171, 17)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
